# Sistema de Recomendacion de peliculas

<img src="PORTADA.png">

# Importaciones

In [1]:
import numpy as np
import pandas as pd
import ipywidgets as widgets
import matplotlib.pyplot as plt

from IPython.display import display

%matplotlib inline

# Dataset de las peliculas

In [2]:
movies = pd.read_csv('movies.csv')

In [3]:
movies.tail()

,movieId,title,genres
9120,162672,Mohenjo Daro (2016),Adventure|Drama|Romance
9121,163056,Shin Godzilla (2016),Action|Adventure|Fantasy|Sci-Fi
9122,163949,The Beatles: Eight Days a Week - The Touring Y...,Documentary
9123,164977,The Gay Desperado (1936),Comedy
9124,164979,"Women of '69, Unboxed",Documentary


# Dateset de Ratings

In [4]:
ratings = pd.read_csv('ratings.csv')
ratings = ratings.drop(['timestamp'], axis=1)

In [5]:
ratings.tail()

,userId,movieId,rating
99999,671,6268,2.5
100000,671,6269,4.0
100001,671,6365,4.0
100002,671,6385,2.5
100003,671,6565,3.5


# Unir los dataSets

In [6]:
moviess = pd.merge(movies, ratings, on= 'movieId')

In [49]:
moviess.tail()

,movieId,title,genres,userId,rating
99999,161944,The Last Brickmaker in America (2001),Drama,287,5.0
100000,162376,Stranger Things,Drama,73,4.5
100001,162542,Rustom (2016),Romance|Thriller,611,5.0
100002,162672,Mohenjo Daro (2016),Adventure|Drama|Romance,611,3.0
100003,163949,The Beatles: Eight Days a Week - The Touring Y...,Documentary,547,5.0


In [8]:
moviess.dropna(inplace=True)

In [9]:
moviess = pd.concat([moviess["genres"].str.get_dummies(sep="|"),moviess[["userId"]],moviess[["movieId"]],moviess[["rating"]]],axis=1)

In [10]:
moviess.tail()

,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,...,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,userId,movieId,rating
99999,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,287,161944,5.0
100000,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,73,162376,4.5
100001,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,611,162542,5.0
100002,0,0,1,0,0,0,0,0,1,0,...,0,0,1,0,0,0,0,611,162672,3.0
100003,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,547,163949,5.0


# Entrenamiento

In [11]:
from sklearn.preprocessing import MaxAbsScaler

In [12]:
max_abs_scaler = MaxAbsScaler()
moviess = max_abs_scaler.fit_transform(moviess)

In [13]:
from sklearn.neighbors import NearestNeighbors

In [14]:
nbrs = NearestNeighbors(n_neighbors=6, algorithm='ball_tree').fit(moviess)

In [15]:
distances, indices = nbrs.kneighbors(moviess)

In [16]:
def get_index(title):
    return moviess[moviess["title"]==title].index.tolist()[0]

In [17]:
def recomendar_movies(query=None,id=None):
    if id:
        for id in indices[id][1:]:
            print(moviess.ix[id]["title"])
    if query:
        found_id = get_index(query)
        for id in indices[found_id][1:]:
            print(moviess.ix[id]["title"])

# Evaluación

In [52]:
recomendar_movies(id=get_index('Ant-Man (2015)'))

Avengers: Age of Ultron (2015)
Independence Day: Resurgence (2016)
Star Trek 3 (2016)
Ant-Man (2015)
Guardians of the Galaxy (2014)
